# Starter example for pyicon

Some specific commands for Jupyter:

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

Load necessary modules:

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import sys
import pyicon as pyic

## Names and paths

Define the name of the simulation, its grid, the path to the data and paths to the grid:

In [3]:
run = 'nib0004'
runname = 'icon_08'
gname = 'r2b6'
lev = 'L64'

path_data     = f'/mnt/lustre01/work/mh0033/m300602/proj_vmix/icon/{runname}/icon-oes/experiments/{run}/'
path_grid     = f'/mnt/lustre01/work/mh0033/m300602/icon/grids/{gname}/'
path_ckdtree  = f'{path_grid}ckdtree/'
fpath_fx      = f'{path_grid}{gname}_{lev}_fx.nc'

## Pyicon IconData object

Create a pyicon IconData object. This object contains information about the grid, and the time steps which are in the given data directory.

In [4]:
IcD = pyic.IconData(
               fname        = run+'_????????????????.nc',
               path_data    = path_data,
               path_grid    = path_grid,
               gname        = gname,
               lev          = lev,
               #path_ckdtree = path_ckdtree,
               #section_name = '',
               #fpath_fx     = fpath_fx,
               do_triangulation = True,
               omit_last_file   = False
              )

## Load the data

Specify some indices (it for time and iz for depth) that should be used to load a subset of the data (one level at one time step):

In [5]:
# --- specify time step
it = np.argmin(np.abs(IcD.times-np.datetime64('2295-01-01T00:00:00')))
# --- specify depth level
iz = np.argmin(np.abs(IcD.depthc-100.))

Finally load the data (in this case temperature):

In [6]:
f = Dataset(IcD.flist_ts[it], 'r')
to = f.variables['to'][IcD.its[it],iz,:]
to_3d = f.variables['to'][IcD.its[it],:,:]
f.close()

Mask land values (assuming 0 data values to be land):

In [7]:
to[to==0.0] = np.ma.masked

We can calculate the mean value:

In [8]:
to.mean()

14.134106462480295

or get information about the size of the data:

In [9]:
to.shape

(235403,)

In [10]:
to_3d.shape

(64, 235403)

## Plotting on the original triangular grid

First mask land values (simply assuming that 0 data values are land):

In [11]:
mask_land = to.mask
# The mask needs to be applied to IconData triangulation object. 
# It is important to add the mask to the IcD.mask_bt mask which masks longitudinally periodic triangles.
IcD.Tri.set_mask(IcD.mask_bt+mask_land)

In [ ]:
hca, hcb = pyic.arrange_axes(1,1, plot_cb=True, sasp=0.5, fig_size_fac=3)
ii=-1

ii+=1; ax=hca[ii]; cax=hcb[ii]
pyic.trishade(IcD.Tri, to, ax=ax, cax=cax, clim='auto')
ax.set_xlabel('longitude')
ax.set_ylabel('latitude')
ax.set_title('temperature at %s and %.1fm depth'%(IcD.times[it], IcD.depthc[iz]))
cax.set_title('$^o$C')

## Interpolate data to a regular grid

Use the pyicon interp_to_rectgrid function. The path to the regular grid ckdtree file (fpath_ckdtree) needs to be specified. It decides which regular grid is taken but only those grids can be chosen which were calculated and saved externally.

In [ ]:
fpath_ckdtree = f'{path_grid}ckdtree/rectgrids/{gname}_res0.30_180W-180E_90S-90N.npz'
lon, lat, toi = pyic.interp_to_rectgrid(to, fpath_ckdtree, coordinates='clat clon')

In [ ]:
hca, hcb = pyic.arrange_axes(1,1, plot_cb=True, sasp=0.5, fig_size_fac=3)
ii=-1

ii+=1; ax=hca[ii]; cax=hcb[ii]
pyic.shade(lon, lat, toi, ax=ax, cax=cax, clim='auto')
ax.set_xlabel('longitude')
ax.set_ylabel('latitude')
ax.set_title('temperature at %s and %.1fm depth'%(IcD.times[it], IcD.depthc[iz]))
cax.set_title('$^o$C')